In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mcu-movies-dataset/rating.csv
/kaggle/input/mcu-movies-dataset/box_office.csv
/kaggle/input/mcu-movies-dataset/creators.csv


# Install Dash
- Dash Bootstrap Components
- Dash Bootstrap Templates

In [7]:
pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 12.6 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install dash-bootstrap-components

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 936.8 kB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install dash-bootstrap-templates -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 430.3 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


# Import Libraries

In [111]:
import pandas as pd
import numpy as np
import dash
from dash.dependencies import Input, Output, State
from dash_table import DataTable
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import qualitative

### Instantiate dash app

In [3]:
dbc_css = ("https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates@V1.0.2/dbc.min.css")
app = dash.Dash(__name__, 
                external_stylesheets=[dbc.themes.SUPERHERO, dbc_css]
               )

# Learn more about external stylesheets templates 
# https://dash-bootstrap-components.opensource.faculty.ai/docs/themes/
# https://github.com/AnnMarieW/dash-bootstrap-templates
# we are using the css theme superhero 
# Preview sample can be found here: https://dash-bootstrap-components.opensource.faculty.ai/docs/themes/explorer/

## Load Data

In [98]:
BOX_OFFICE = pd.read_csv('/kaggle/input/mcu-movies-dataset/box_office.csv')
RATING = pd.read_csv('/kaggle/input/mcu-movies-dataset/rating.csv')

#### Data Cleaning

In [99]:
BOX_OFFICE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 9 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   Phase                               30 non-null     object
 1   Film                                30 non-null     object
 2   U.S. release date                   30 non-null     object
 3   Box office gross U.S. and Canada    30 non-null     int64 
 4   Box office gross Other territories  30 non-null     int64 
 5   Box office gross Worldwide          30 non-null     int64 
 6   All-time ranking U.S. and Canada    30 non-null     int64 
 7   All-time ranking Worldwide          30 non-null     int64 
 8   Budget                              30 non-null     object
dtypes: int64(5), object(4)
memory usage: 2.2+ KB


In [100]:
# rename columns
BOX_OFFICE.rename(columns={'U.S. release date': 'Release Date',
                           'Box office gross U.S. and Canada': 'North America Box Office',
                           'Box office gross Other territories': 'Others Box Office', 
                           'Box office gross Worldwide': 'Worldwide Box Office',
                           'All-time ranking U.S. and Canada': 'Ranking US',
                           'All-time ranking Worldwide': 'Ranking Worldwide'}, 
                      inplace=True)

In [101]:
#remove '$' and replace 'million' with '000000'
BOX_OFFICE['Budget'] = BOX_OFFICE['Budget'].str.replace('$', '', regex=True)

# remove 'space'
BOX_OFFICE['Budget'] = BOX_OFFICE['Budget'].str.replace('\s+', '', regex=True)

#remove million
BOX_OFFICE['Budget'] = BOX_OFFICE['Budget'].str.replace('million', '000000', regex=True)

#change 'Budget' column to numeric
BOX_OFFICE['Budget'] = pd.to_numeric(BOX_OFFICE['Budget'])

In [102]:
#change 'date' column to datetime series

BOX_OFFICE['Release Date'] = pd.to_datetime(BOX_OFFICE['Release Date'])

In [103]:
BOX_OFFICE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Phase                     30 non-null     object        
 1   Film                      30 non-null     object        
 2   Release Date              30 non-null     datetime64[ns]
 3   North America Box Office  30 non-null     int64         
 4   Others Box Office         30 non-null     int64         
 5   Worldwide Box Office      30 non-null     int64         
 6   Ranking US                30 non-null     int64         
 7   Ranking Worldwide         30 non-null     int64         
 8   Budget                    30 non-null     int64         
dtypes: datetime64[ns](1), int64(6), object(2)
memory usage: 2.2+ KB


In [104]:
BOX_OFFICE.head(5)

,Phase,Film,Release Date,North America Box Office,Others Box Office,Worldwide Box Office,Ranking US,Ranking Worldwide,Budget
0,Phase One,Iron Man,2008-05-02,319034126,266762121,585796247,79,176,140000000
1,Phase One,The Incredible Hulk,2008-06-13,134806913,129964083,264770996,465,593,150000000
2,Phase One,Iron Man 2,2010-05-07,312433331,311500000,623933331,85,158,200000000
3,Phase One,Thor,2011-05-06,181030624,268295994,449326618,262,268,150000000
4,Phase One,Captain America: The First Avenger,2011-07-22,176654505,193915269,370569774,279,367,140000000


In [105]:
RATING.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Phase                     30 non-null     object
 1   Film                      30 non-null     object
 2   Critical Rotten Tomatoes  30 non-null     object
 3   Critical Metacritic       30 non-null     object
 4   Public CinemaScore        30 non-null     object
 5   Public PostTrak           30 non-null     object
dtypes: object(6)
memory usage: 1.5+ KB


In [106]:
# rename columns

RATING.rename(columns={'Critical Rotten Tomatoes': 'Rotten Tomatoes Rating',
                       'Critical Metacritic': 'Metacrtic Rating',
                       'Public CinemaScore': 'CinemaScore Grade',
                       'Public PostTrak': 'PostTrak Rating', 
                       }, 
                      inplace=True)

In [108]:
#remove '% (XX reviews)' in Rotten Tomatoes

RATING['Rotten Tomatoes Rating'] = RATING['Rotten Tomatoes Rating'].str[:-15]

# remove '% (XX reviews)' in MetaCritic
RATING['Metacrtic Rating'] = RATING['Metacrtic Rating'].str[:-13]


In [110]:
RATING.head(5)

,Phase,Film,Rotten Tomatoes Rating,Metacrtic Rating,CinemaScore Grade,PostTrak Rating
0,Phase One,Iron Man,94,79,A,—
1,Phase One,The Incredible Hulk,67,61,A−,—
2,Phase One,Iron Man 2,71,57,A,—
3,Phase One,Thor,77,57,B+,—
4,Phase One,Captain America: The First Avenger,79,66,A−,—


**To be continue...**


The goal of this project is to use Dash and Plotly to create an HTML interactive Dashboard 